# `bw_timex` Teaching Exercise - Time-explicit LCA of an Electric Vehicle vs Petrol Vehicle (SOLUTION NOTEBOOK)

In this exercise notebook you will compare the climate performance of a battery electric vehicle (BEV) and an internal combustion engine car (ICEC), in a time explicit way, using different decarbonisation scenarios. 

In order to have a fair comparison**, we will start from two existing Ecoinvent activities (listed below) and will add the temporal information as needed. This notebook will guide you through the steps needed to build the model. But if you are unsure, you can either ask the instructors or have a look at the solution notebook (Note that this just provides one solution, and there are often multiple ways to get to the same result, so we encourage you to use the way that is most intuitive to you!). 

At the end of this notebook, there are a few bonus questions. 


> ** Warning: Any LCA is only every as good as the underlying data. the data on cars in Ecoinvent is really outdated (check the description/comment of the activities to get an idea about the data), so the results from this exercise should be interpreted with care, as should any LCA result be of course!

In [ ]:
import bw2data as bd

bd.projects.set_current("Timex_teaching_ei310")

## Prospective background databases

In addition to the very optimistic PkBudg650 scenario used in the previous notebook, we'll also consider a 'national policies implemented' (NPi) as a business as usual (BAU) scenario here:

- REMIND-EU - SSP2-NPi       : BAU scenario with ~3˚ warming by 2100
- REMIND-EU - SSP2-PkBudg650 : Optimistic scenario in line with Paris 2015 with 1.5˚-1.9˚ warming by 2100

> See [the Premise documentation](https://premise.readthedocs.io/en/latest/introduction.html#default-iam-scenarios) for a brief overview of the IAM scenarios

Let's get easy handles for the databases for ease of use:

National Policies and Implemention (NPi) scenario:

In [ ]:
db_2020_NPi = bd.Database("ei_cutoff_3.10_remind-eu_SSP2-NPi_2020 2025-10-08")
db_2030_NPi = bd.Database("ei_cutoff_3.10_remind-eu_SSP2-NPi_2030 2025-10-08")
db_2040_NPi = bd.Database("ei_cutoff_3.10_remind-eu_SSP2-NPi_2040 2025-10-08")
db_2050_NPi = bd.Database("ei_cutoff_3.10_remind-eu_SSP2-NPi_2050 2025-10-08")

Peak-Budget 650 scenario:

In [ ]:
db_2020_PkBudg650 = bd.Database("ei_cutoff_3.10_remind-eu_SSP2-PkBudg650_2020 2025-10-08")
db_2030_PkBudg650 = bd.Database("ei_cutoff_3.10_remind-eu_SSP2-PkBudg650_2030 2025-10-08")
db_2040_PkBudg650 = bd.Database("ei_cutoff_3.10_remind-eu_SSP2-PkBudg650_2040 2025-10-08")
db_2050_PkBudg650 = bd.Database("ei_cutoff_3.10_remind-eu_SSP2-PkBudg650_2050 2025-10-08")

## Case study setup

In this exercise we will go about setting up the forground system in a slightly different way. In our case, driving a(n electric) car is already represented in ecoinvent. So all we would need to do is to add the temporal distributions to the exchanges. 

In the current implementation of `bw_timex`, all activities containing temporal information on the exchanges need to 'live' in the foreground system, so that their inputs can get linked to different background activities.

Let's start the definition of our functional unit (FU): 

- Driving a passenger car for 200,000km (including production, EoL treatment, maintenance, etc), over a period of 15 years.


### Modeling the production system

As a first step of modelling the production system, let's create a new foreground database:


In [ ]:
fg_db_name = "foreground_bev_vs_icec"

if fg_db_name in bd.databases:
    del bd.databases[fg_db_name] # to make sure we create the foreground from scratch

In [ ]:
foreground = bd.Database(fg_db_name)
foreground.register()

And let's define the FU relevant quantities: 

In [ ]:
LIFETIME_KM = 200000.  # lifetime of the vehicle in km
LIFETIME = 15  # lifetime of the vehicle in years
ANNUAL_KM = LIFETIME_KM / LIFETIME  # annual km

Now let's create two activities in the foreground that represent the functional units as defined above (production/driving/EoL treatment of a car for 200,000km over 15 years). We can call them LC_BEV and LC_ICEC for example (LC=life cyle).

In [ ]:
LC_BEV = foreground.new_node("lc_bev", name="Life cycle of a battery electric vehicle (BEV)", unit="unit")
LC_BEV['reference product'] = "LC_BEV"
LC_BEV.new_edge(input=LC_BEV, amount=1, type='production')
LC_BEV.save()

LC_ICEC = foreground.new_node("lc_icec", name="Life cycle of a internal combustion engine car (ICEC)", unit="unit")
LC_ICEC['reference product'] = "LC_ICEC"
LC_ICEC.new_edge(input=LC_ICEC, amount=1, type='production')
LC_ICEC.save()

Let's make a simplified overview of the activities at every step (example for the ICEC). So far, we only have defined the life cycle activity
in the foreground. We will use different colours for the foreground and background processes. You can also do this on paper.

```mermaid
flowchart LR
    LC_ICEC(Life Cycle ICEC):::fg

    classDef ei color:#222832, fill:#3fb1c5, stroke:none;
    classDef fg color:#222832, fill:#9c5ffd, stroke:none;

```
Legend:
```mermaid
flowchart TD
    foreground(Foreground process):::fg 
    background(Background process):::ei

    classDef ei color:#222832, fill:#3fb1c5, stroke:none;
    classDef fg color:#222832, fill:#9c5ffd, stroke:none;

```


Ecoinvent already contains activities that represent transport with a passenger car. So let's get the following two activities from the base database** for the BAU scenario and create a copy in our foreground database:

- Driving a ICEC: "transport, passenger car, gasoline, Medium SUV, EURO-6"
- Driving a BEV: "transport, passenger car, gasoline, Medium SUV, EURO-6"

The idea is to first see how these activities are set up. It might help to draw your own system definition to create an overview of the exchanges and see to which exchanges you want to add temporal information. The following steps guide you through the process. 

HINT: There are multiple locations of this activity in the premise database, choose one of your liking/or that makes sense for you.
HINT: Copy to a different database by first copying the activity and then changing the database.

> ** We consider 2020 as our base year and db_2020 as the 'base' database

In [ ]:
# only run this cell once!
driving_icec = db_2020_NPi.get(name='transport, passenger car, gasoline, Medium SUV, EURO-6', location="RER").copy()
driving_icec['database'] = fg_db_name
driving_bev = db_2020_NPi.get(name='transport, passenger car, battery electric, Medium SUV', location="RER").copy()
driving_bev['database'] = fg_db_name

The reference product of these activities should be 1 km traveled by the respective vehicle, but let's check the reference product and its unit:

In [ ]:
print(f'Driving BEV reference product: {driving_bev.rp_exchange()}')
print(f'Driving BEV rp unit: {driving_bev.rp_exchange()["unit"]}')
print(f'Driving ICEC reference product: {driving_icec.rp_exchange()}')
print(f'Driving ICEC rp unit: {driving_icec.rp_exchange()["unit"]}')

Have a look at what the inputs into such a car are and identify the relevant exchanges:

In [ ]:
list(driving_bev.technosphere())

In [ ]:
list(driving_icec.technosphere())

Let's make a simplified overview of these activities so far (example for the ICEC): 

```mermaid
flowchart LR
    passenger_icec(Passenger car, gasoline, ...):::ei-->transport_icec
    petrol(Market for Petrol, ...):::ei-->transport_icec
    maintenance_wear(Maintenance and wear):::ei-->transport_icec
    transport_icec(Driving an ICEC):::fg -.-> lc_icec(Life Cycle ICEC):::fg
    
    

    classDef ei color:#222832, fill:#3fb1c5, stroke:none;
    classDef fg color:#222832, fill:#9c5ffd, stroke:none;
```

Where the dotted line indicates that we have not made this connection yet. 

These 'transport activities' include both the car, which in itself contains both production and EoL treatment (check this!!), as well as the fuel/electricity, maintenance and wear etc. 

As we want to add temporal information to the car process (production and EoL), we need to move a copy of this process into the foreground.
Here, we need to remember to set the original input of the car process from the 'base' database into the driving activity to 0 to avoid double counting.

In [ ]:
# first copy the activities production/EoL treatment of the cars into the foreground
bev_production_eol = db_2020_NPi.get(name='passenger car, battery electric, Medium SUV').copy()
bev_production_eol['database'] = fg_db_name

icec_production_eol = db_2020_NPi.get(name='Passenger car, gasoline, Medium SUV, EURO-6d').copy()
icec_production_eol['database'] = fg_db_name


Have a look at the exchanges of the production/EoL activity and see that these indeed include both production and EoL.


In [ ]:
for exc in bev_production_eol.technosphere():
    print(exc)

In [ ]:
# now set the input of the car into the driving activity to 0 to avoid double counting
for exc in driving_bev.technosphere():
    if exc.input['name'] == 'passenger car, battery electric, Medium SUV':
        exc.delete()
        break
for exc in driving_icec.technosphere():
    if exc.input['name'] == 'Passenger car, gasoline, Medium SUV, EURO-6d':
        exc.delete()
        break

So now update our overview:

```{mermaid}
flowchart LR
    petrol(Market for Petrol, ...):::ei-->transport_icec
    maintenance_wear(Maintenance and wear):::ei-->transport_icec
    transport_icec(Driving an ICEC):::fg -.-> lc_icec(Life Cycle ICEC):::fg
    passenger_icec(Passenger car, gasoline, ...):::fg-.->lc_icec
    production_icec(Production ICEC):::ei-->passenger_icec
    eol_icec(EoL treatment ICEC):::ei-->passenger_icec
    

    classDef ei color:#222832, fill:#3fb1c5, stroke:none;
    classDef fg color:#222832, fill:#9c5ffd, stroke:none;
```

Where the dotted line indicates that we have not made this connection yet. 

### Adding the relevant exchanges and temporal information

Now it is time to add the relevant exchanges to the functional unit activities together with relevant temporal distributions. Be careful to add the temporal distribution to production and EoL at the right exchange, so that production happens before the use phase and EoL after the use phase. 

Tipp: for ease of use, or if you forgot how to define temporal distributions you can use the interactive_td_widget!

Tipp 2: Use the utility function add_temporal_distribution_to_exchange to add the temporal distribution to the EoL of the production/EoL activity of the cars. 


Let's first draw the system we want including where we want TD's


```mermaid
flowchart LR
    petrol(Market for Petrol, ...):::ei-->transport_icec
    maintenance_wear(Maintenance and wear):::ei-->transport_icec
    transport_icec(Driving an ICEC):::fg ==>|Temporal information|lc_icec(Life Cycle ICEC):::fg
    passenger_icec(Passenger car, gasoline, ...):::fg==>|Temporal information|lc_icec
    production_icec(Production ICEC):::ei-->passenger_icec
    eol_icec(EoL treatment ICEC):::ei==>|Temporal information|passenger_icec
    

    classDef ei color:#222832, fill:#3fb1c5, stroke:none;
    classDef fg color:#222832, fill:#9c5ffd, stroke:none;
```




In [ ]:

LC_BEV.new_edge(input=driving_bev, amount=LIFETIME_KM, type='technosphere').save()
LC_BEV.new_edge(input=bev_production_eol, amount=1, type='technosphere').save()


LC_ICEC.new_edge(input=driving_icec, amount=LIFETIME_KM, type='technosphere').save()
LC_ICEC.new_edge(input=icec_production_eol, amount=1, type='technosphere').save()

Now define and add the temporal distributions to the relevant exchanges.

Some relevant imports:

In [ ]:
import numpy as np

from bw_timex.utils import add_temporal_distribution_to_exchange
from bw_temporalis import TemporalDistribution, easy_timedelta_distribution

We came up with the following temporal information:
* for the use phase, we assume a uniform distribution over the lifetime of the car (15 years)
* for the production of the cars, we assume a normal distribution over 4 years prior to use
* for the EoL, a fixed point in time at the end of the lifetime + 1 year

In [ ]:
td_driving =  easy_timedelta_distribution(
    start=0,
    end=15,
    resolution='Y',
    steps=16,
    kind='uniform'
)

td_production = easy_timedelta_distribution(
    start=-4,
    end=-1,
    resolution='Y',
    steps=4,
    kind='normal',
    param = 1
)


date = np.array([16], dtype='timedelta64[Y]')
amount = np.array([1], dtype=float)
td_eol = TemporalDistribution(date=date, amount=amount)


add_temporal_distribution_to_exchange(td_driving,
                                      input_node=driving_bev,
                                      output_node=LC_BEV
                                      )
add_temporal_distribution_to_exchange(td_driving,
                                      input_node=driving_icec,
                                      output_node=LC_ICEC
                                      )

Now we add the temporal distributions for production and end of life to the exchanges of the production/EoL activities of the cars. Here we will make use of the fact that in ecoinvent waste treatment activities are modelled as negative inputs.

In [ ]:
for exc in bev_production_eol.technosphere():
    if exc['amount'] >= 0:  # production
        add_temporal_distribution_to_exchange(td_production,
                                              input_node=exc.input,
                                              output_node=exc.output)
    else:  # EoL
        add_temporal_distribution_to_exchange(td_eol,
                                              input_node=exc.input,
                                              output_node=exc.output)

In [ ]:
for exc in icec_production_eol.technosphere():
    if exc['amount'] >= 0:  # production
        add_temporal_distribution_to_exchange(td_production,
                                              input_node=exc.input,
                                              output_node=exc.output)
    else:  # EoL
        add_temporal_distribution_to_exchange(td_eol,
                                              input_node=exc.input,
                                              output_node=exc.output)

### Checking all went right

Before calculating the time-explicit LCA, make sure that everything went right so far. As a check, the standard LCA score of the original transport activity with the corresponding amount should be equal the standard LCA of the modified life cycle activity created in the foreground!

In [ ]:
import bw2calc as bc

# standard LCA of the original transport activity with the corresponding amount
transport_icec= db_2020_NPi.get(name='transport, passenger car, gasoline, Medium SUV, EURO-6', location="RER")
method = ('ecoinvent-3.10', 'EF v3.1', 'climate change', 'global warming potential (GWP100)')
lca = bc.LCA({transport_icec: LIFETIME_KM}, method)
lca.lci()
lca.lcia()

# standard LCA of the modified life cycle activity created in the foreground
lca_fg = bc.LCA({LC_ICEC: 1}, method)
lca_fg.lci()
lca_fg.lcia()

assert(np.isclose(lca.score, lca_fg.score)) # if this passes, it's all good

In [ ]:
# now the same for the BEV
# standard LCA of the original transport activity with the corresponding amount
transport_bev = db_2020_NPi.get(name='transport, passenger car, battery electric, Medium SUV', location="RER")
lca_bev = bc.LCA({transport_bev: LIFETIME_KM}, method)
lca_bev.lci()
lca_bev.lcia()

# standard LCA of the modified life cycle activity created in the foreground
lca_bev_fg = bc.LCA({LC_BEV: 1}, method)
lca_bev_fg.lci()
lca_bev_fg.lcia()

assert(np.isclose(lca_bev.score, lca_bev_fg.score)) # if this passes, it's all good

## Time-explicit LCA with `bw_timex`

Now we can start using `bw_timex` to build the process timeline, build the time-explicit inventory and calculate the time explicit scores.

Start by creating a dictionary mapping the respective databases to the relevant timestamps. 
Then instantiate your timexLCA object, build the timeline, calculate the LCI, and the LCIA for both options.


In [ ]:
from datetime import datetime

database_dates = {
    db_2020_NPi.name: datetime.strptime("2020", "%Y"),
    db_2030_NPi.name: datetime.strptime("2030", "%Y"),
    db_2040_NPi.name: datetime.strptime("2040", "%Y"),
    db_2050_NPi.name: datetime.strptime("2050", "%Y"),
    foreground.name: "dynamic", # flag databases that should be temporally distributed with "dynamic"
}

In [ ]:
from bw_timex import TimexLCA

tlca_BEV = TimexLCA({LC_BEV: 1}, method, database_dates)
tlca_BEV.build_timeline(temporal_grouping="year")  # build timeline with yearly steps

In [ ]:
tlca_BEV.lci()

In [ ]:
tlca_BEV.dynamic_lcia(metric="GWP", time_horizon=100, fixed_time_horizon=True)  # calculate the dynamic LCIA for GWP100

In [ ]:
print(f'Total: {tlca_BEV.dynamic_score:.2e} kg CO2-eq for {LIFETIME_KM} km driving with BEV')

In [ ]:
tlca_ICEC = TimexLCA({LC_ICEC: 1}, method, database_dates)
tlca_ICEC.build_timeline(temporal_grouping="year")  # build timeline with yearly steps
tlca_ICEC.lci()  # calculate the dynamic inventory
tlca_ICEC.dynamic_lcia(metric="GWP", time_horizon=100, fixed_time_horizon=True)

In [ ]:
print(f'Total: {tlca_ICEC.dynamic_score:.2e} kg CO2-eq for {LIFETIME_KM} km driving with ICEC')

## Visualise results and compare the different cars and scenarios

- Now find a good way to visualise the results. There are some simple plotting functions included such as plot_dynamic_inventory()

- Compare the the time-explicit results with the static and prospective cases

- Try another base year (Hint: you can just change the starting year)

- Run the other scenario to see the differences for both cars 



In [ ]:
# As a first try we can use the integrated plotting function of timex
tlca_BEV.plot_dynamic_characterized_inventory(sum_emissions_within_activity=True)
tlca_ICEC.plot_dynamic_characterized_inventory(sum_emissions_within_activity=True)

## BONUS options:

- Add temporal information further throughout the supply chain. To do this, move further background processes to the foreground and add TDs, e.g., for the steel that is needed for the vehicle production processes.

- Use temporal distributions to model a fleet (of changing composition). (Hint: for a fleet some cars will live longer, some shorter. This can be modelled using a temporal distribution for the use phase and the EoL)
    - Now you can also look at different impacts (e.g. metal demand for example and see the metal demand per year needed to build a fleet)

- How does the comparison change if we replace the petrol consumption of the car with biofuel (e.g. this activity:  'petrol production, 85% ethanol by volume from biomass' (kilogram, CH, None)). And then what about if you include biogenic carbon emissions by using for example this method: ('ecoinvent-3.10',
  'IPCC 2021',
  'climate change: biogenic',
  'global warming potential (GWP100)')

- Use dynamic_lcia(use_disaggregated_lci=TRUE) to perform a contribution analysis on the background activities contributing to the impacts. (Note that this function can take a while), find a good way to visualize this.